# 1. Motivation
## 1.1 Introduction
Reddit is a large forum visited by millions of people every day, where all content is user currated. To structure this vast website, reddit is seperated into sub forums called subreddits and there exists a subreddit for almost everything imagineable. These subreddits are created by the users and are therefore not a static set of categories like you might see on a news website. A user can subscribe to a set of subreddits to customize the posts it will receive. The user can create posts and comment on posts by other users. This project will analyze the activity pattern of reddit users to find meta communities between subreddits. If you wish to explore one of the subreddits we mention, then you can access any subreddit by going to *www.redddit.com/r/'NameOfSubreddit'*  For a video explination of how reddit works, watch this execellent video by CPG Grey: https://www.youtube.com/watch?v=tlI022aUWQQ

This notebook will contain the important code snippits for the project, but not complete blocks of runnable code. 

## 1.2 The dataset
Due to the immense scale of Reddit, some compromises have had to be made in data selection and data processing. 
The data of this project is based in the month of september 2017 and consists of posts and comments for this time period. It was chosen to limit the data to a single month to make data processing somewhat manageable, as the entire dataset is multiple terabytes. September was chosen because reddit inflates a lot during the summer month, due to the influx of users, with a lot of time on their hands. As reddit is a website many of its users visit many times a week, we asseded that one month of data would be sufficient for the analysis. 

A comment data point contains the text body, author, score and the name of the subreddit it was submitted to. 
A post data point contains the title, author, score, whether the post was marked 'not safe for work(NSFW)' and the subreddit it was submitted to. 
        
The dataset of comments is used to analyse the activity of the subreddits. The activity of a subreddit can be quantified by the different users that submit comments to it. If two subreddits have a large intersections of users, a connection is found. 
The dataset of posts are primarily used in the text analysis. The titles of each post within a community is used to compare content. The original intent was to use the text body of the comment for this, but this approach was chosen to limit the amount of data to process. 


## 1.3 Goal
The goal with this project is to find affiliations across the different subreddits through their active users. The hypothesis is that otherwise unrelated subreddits are affiliated unbeknownst to their users, making way for interesting results about the reddit community. This project will mostly revolve about the communities and how they were found. Further analysis could be done on the outputs of specific meta communities to determine traits in their collective reaction to humor, controversy or external sources used in posts. 

# 2. Basic stats
## 2.1 Initial data processing and cleaning
The Reddit network created in this project is build from the dataset of comments which is approximately 17GB. Note that more than half of this data is the comment text body. In the inital processing of the data, each subreddit name is replaced with an unique integer and a dictionary is created for later reference. This is done to increase the speed of comparisons and decrease memory ussage. A set of comment authors(users) are assosiated with each subreddit, meaning if you submitted a comment to the given subreddit, you will appear in the set. All combinations of subreddits are iterated and if a combination have intersecting users, an undirected edge will be saved between these. The size of this intersection dictates the wieght of this edge, which will be used in later processing. This process is very intensive as the amount of combinations are equal to number of subreddits squared. This was not manageable with a syncronous python script, so a multi threaded java program was written to compute this. The  program can be found here: https://github.com/ageil/redditgraph/tree/master/Java


**Basic statistics for full network and datasets:**

Comment dataset(September 2017): 
 * Size: 16.7 GB
 * Rows(amount of comments): 77.510.000
 * Total authors(users): 15.108.000
 * Total subreddits: 70.888
 
Post dataset(September 2017):
 * Size: 1 GB
 * Rows(amount of posts): 9800000
 
Full network:
 * Nodes: 70.888 (Subreddits)
 * Edges: 225.777.872
 * Avg. degree: 2999 
 * Max degree: 47547 (For subreddit 'AskReddit') 
 * Avg. edge weight: 2.6 (Average amount of users in common)
 
 
Top 10 subreddits by degree:
1. AskReddit: 47547
2. pics: 38281
3. funny: 37812
4. todayilearned: 36951
5. videos: 35692
6. mildlyinteresting: 35615
7. worldnews: 35165
8. gifs: 34105
9. gaming: 32683
10. Showerthoughts: 32453
 
The graph below displays the weight destribution with logged axis.
<h1><center>Weight distribution of full network</center></h1>
![title](https://i.imgur.com/EbwloGi.png)
 
## 2.2 Statistic discussion
When you create a Reddit account you will automatically be subscribed to a default set of the most popular subreddits. It is therefore assumed that these subreddits will be among those with the highest degree. This is confirmed by the fact that all subreddits in the top 10 above is default subreddits. 


It is clear by the distribution of edge weights(users in common) that there are a lot of nodes(subreddits) with few users in common. Some of these edges will have to be removed in order to make further processing meaningful, the upcomming section will explain how the decision of where to cut the network was made. 
 


# 3. Tools, theory and analysis.
## 3.1 Community detection


## 3.2 Working with text
Because of the scale of this project, it was not possible to go very deep with text analysis as this caused excessive program running times. The text used will consist of all post titles from the network cut. The main focus of the this analysis is the text similarity in a community.  The following hypothesis is proposed:
> H1: Text similarity will be higher inside a community when compared to the entire network.

This hypthosis will be investigated using term frequency-inverse document frequency. This method reflects on how important a word is for a given document. All titles are first cleaned for stopwords, punctuation, digits, duplicates and finally are all words converted to lower case. Punctuation is removed using a regular expression and text is parsed to the method with utf-8 encoding. The method below is used:

In [ ]:
def cleanup(text):
    stopws = set(stopwords.words('english'))
    
    tokens = nltk.RegexpTokenizer(r'\w+').tokenize(text) # strip punctuation
    tokens = [token for token in tokens if token.lower() not in stopws] # strip stopwords
    tokens = [token for token in tokens if not token.isdigit()] # strip digits
    tokens = list(set(tokens)) # remove duplicates
    clean_text = " ".join(tokens)
    
    return clean_text

When using TF-IDF the documents and corpus will have to be defined. Each subreddit will represent a single document that is the concatination of all post titles associated with that subreddit. The corpus will be collection of all documents associated with the subreddits in network cut. A TF-IDF vectorizer will is used to generate a matrix of the frequency for the different words in each text. The cosine similarity between the different vectors will be found so it is possible to compare two subreddits. The code below uses the *sklearn* module to do these computations:

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
tfidf = TfidfVectorizer().fit_transform(textDictFlat.values())
cossim = cosine_similarity(tfidf)

The cossim matrix is of size 12083 by 12083 hence representing the amount of subreddits on each axis. To compare similarity within communities, the pairwise similary of subreddits in the community is found. The mean of these pairs represents the text similarity in the community. The mthod below is used to compute this mean.

In [ ]:
import numpy as np
def meanCossimForCommunity(i):
    subReddits = []
    indicies = []
    # Find subreddits in group
    for key, val in groupDict.items():
        if val == i:
            subReddits.append(key)
    print(str(len(subReddits)) + ' subreddits were found for group ' + str(i))
    
    # Find indicies of these subreddits corrosponding to cossim matrix
    for sub in subReddits:
        try:
            indicies.append(textDictOrdered.keys().index(sub))
        except: 
            continue
    print(str(len(indicies)) + ' indicies were found for group ' + str(i)) 
    
    # Iterate all pairs of subreddits within a group
    means = []
    for i in indicies:
        tempMean = []
        for j in indicies:
            if not i == j: # Avoid diagonal of matrix where a sub is compared to itself
                tempMean.append(cossim[i][j])
        means.append(np.mean(tempMean))
        
    return np.mean(means)

Running this method for the four biggest communites yields the following results:
![title](https://i.imgur.com/cO5cCxZ.png)
Most noticeable is that community 2 has a lower similarity than the mean of the entire network. Community 2 contains approx. 65% of the total subreddits(nodes), hence catching a substantially broader varity of text. Community 2 also contains most of the default subreddits which makes the text even less reliable, as many different users post there. These results could be improved if the louvain community algorith was better at splitting this very large community. 

The 3 other communities are all above the total mean, hence containing more similar text. Therefore we fail to reject the hypothesis H1.

# 4. Discussion